In [1]:
import gurobipy as gp
import matplotlib.pyplot as plt
import csv
import re

In [11]:
# Code that may be used to obtain coordinates 
'''
geolocator = Nominatim (user_agent = 'my app')
location = geolocator.geocode("boston, ma") #user defined location
park_number_dict = {0: 'Boston'}
coordinates = [['Boston',location.latitude,location.longitude]]
'''

# import national park data and starting location
# add source (later)
np_filein = open('Updated_input_Boston.csv', 'r', encoding='UTF-8')
csv_reader = csv.reader(np_filein, delimiter=',')
coordinates = []
park_number_dict = {}

park_number_dict = {}
header = next(csv_reader)
counter = 0

for row in csv_reader:
    np_in = [row[2], row[0], row[1]]
    coordinates.append(np_in)
    park_number_dict[counter] = row[2]
    counter += 1
    
np_filein.close()

#print(park_number_dict)
#print(coordinates)

In [12]:
## Variable definitions
travel = {}
park_sequence = {}
subtour = {}

In [60]:
# INPUT PARAMETERS
# Is there a way to print an error if not in list? For future maybe...OR we could do random generator for location on list?

# User-defined
max_trip_length = 10 # days
max_subtrips = 25 #starting point (WARM START!!!!)

# Static
trip_origin = 0
num_parks = len(coordinates)
M = 1e8

# Static but with possible future user-defined option
hours_in_day = 16
length_of_stay = 16 #hours at each park; Future option could include a matrix by destination
flight_time = 16 # Future option could consider pulling flight time from an API and compare with drive time


In [61]:
# Calculate travel time matrix from Google API (file is the output)

np_matrix = open('distancematrix_file_new.csv', 'r', encoding='UTF-8')
csv_reader = csv.reader(np_matrix, delimiter=',')

header = next(csv_reader)

In [62]:
# Read travel times into a matrix table; Each list travel is the list of travel times from an origin
#for i in range(0, len(traveltimes)):
    #travel[i] = traveltimes[i][0:]
#print(travel)

def np_dd_parse(dd_input):
    pattern = r';'
    l = re.split(pattern, dd_input) 
    
    return l

# Read travel times into a dictionary of dictionaries; The origin is the outside dictionary key and the value is a dictionary of travel times to a destination
# For islands, the travel time and duration is set to M value (1e8)
travel = {}
    
np_matrix = open('distancematrix_file_new.csv', 'r', encoding='UTF-8')
csv_reader = csv.reader(np_matrix, delimiter=',')

header = next(csv_reader)
flight_matrix = [] # list of parks that are worth flying to from the origin 
individual_trips = [] # list of special parks that must be excluded from general trip

for row in csv_reader:
    np_dict_internal = {}
    header_count = 0
    park_count = -1
    for np in row:
        
        if header_count > 0:
            if row[0] == park_number_dict[0] or park_count == 0:
                if np_dd_parse(np)[0] == 'N/A':
                    dd_tuple = (-1, flight_time) #-1 long distance, difficult to calculate real route
                    if header_count > 1: 
                        individual_trips.append(header_count-1)
                        flight_matrix.append(header_count-1)
                elif float(np_dd_parse(np)[1]) > flight_time:
                    dd_tuple = (float(np_dd_parse(np)[0]), flight_time)
                    if header_count > 1: 
                        flight_matrix.append(header_count-1) 
                else:
                    dd_tuple = (float(np_dd_parse(np)[0]), float(np_dd_parse(np)[1]))
            else:
                if np_dd_parse(np)[0] != 'N/A':                    
                    dd_tuple = (float(np_dd_parse(np)[0]), float(np_dd_parse(np)[1]))
                else:
                    dd_tuple = (-1, M) #-1 long distance, difficult to calculate real route
            np_dict_internal[header[header_count]] = dd_tuple
        header_count += 1
        park_count += 1
        
    travel[row[0]] = np_dict_internal
 
np_matrix.close()

# print(flight_matrix)
# print(individual_trips) 
# print(travel['Boston']) # Dictionary is Destination: (Distance, Time)

In [64]:
#Model
m = gp.Model("national_park_trips")

#Decision variables
#define Pijk's (Park sequence binary variable)
for trip in range(max_subtrips):
    for origin in range(num_parks):
        for destination in range (num_parks):
            park_sequence[origin,destination,trip] = m.addVar(vtype = gp.GRB.BINARY) 
            

for origin in range(num_parks):      #subtour elimination
    if (origin != 0) and (origin not in individual_trips): # code logic excludes individual trips
        subtour[origin] = m.addVar(lb = 1, ub = num_parks-1*len(individual_trips)) 
    else:
        subtour[origin] = m.addVar()
            
            
# Set objective function; to minimize total time across all subtrips
m.setObjective(sum(sum(sum(park_sequence[origin, destination, trip] * travel[park_number_dict[origin]][park_number_dict[destination]][1]
                           for origin in range(num_parks)) 
                       for destination in range(num_parks)) 
                   for trip in range(max_subtrips)) 
               + length_of_stay*(sum(park_sequence[origin, destination, trip] 
                                     for origin in range(num_parks) 
                                     for destination in range(1,num_parks)
                                     for trip in range(max_subtrips))), gp.GRB.MINIMIZE)

# Constraints
# Must visit each park one time (assumes closed loop, and counting 16 hours only at the destination)
for destination in range(1, num_parks):
    m.addConstr(sum(sum(park_sequence[origin, destination, trip] for origin in range(num_parks)) 
                    for trip in range(max_subtrips)) == 1)

# Leave park if visited in trip
for trip in range(max_subtrips):
    for origin in range(num_parks):
        m.addConstr(sum(park_sequence[origin, destination, trip] for destination in range(num_parks)) == 
                    sum(park_sequence[destination, origin, trip] for destination in range(num_parks)))
        
        # An arc can never have the same origin and destination
        m.addConstr(park_sequence[origin, origin, trip] == 0)

    
# Travel time for each trip cannot exceed the defined limit (trip length in hours)
for trip in range(max_subtrips):
    m.addConstr(sum(sum(park_sequence[origin, destination, trip] * travel[park_number_dict[origin]][park_number_dict[destination]][1] 
                        for origin in range(num_parks)) 
                    for destination in range(num_parks)) 
                <= max_trip_length * hours_in_day 
                - length_of_stay*(sum(park_sequence[origin, destination, trip] 
                                      for origin in range(num_parks) 
                                      for destination in range(1,num_parks))))

# Activate origin if trip is needed
for trip in range(max_subtrips):
    m.addConstr(sum(park_sequence[trip_origin, destination, trip] for destination in range(1,num_parks))<= 1)
    m.addConstr(sum(sum(park_sequence[origin, destination, trip] for origin in range(num_parks)) 
                    for destination in range(num_parks)) 
                <= M*sum(park_sequence[trip_origin, destination, trip] 
                         for destination in range(num_parks)))

# Subtour elimination constraint
for origin in range(1, num_parks):
    for destination in range(1, num_parks):
            if (origin != destination) and (origin not in individual_trips) and (destination not in individual_trips) :
                m.addConstr(subtour[origin] + (num_parks-1)*sum(park_sequence[origin,destination,trip] for trip in range(max_subtrips)) 
                        <= subtour[destination] + (num_parks-2))
                
# Create individual trips for special parks. LOGIC ONLY, NOT in MIP formal formulation
for trip in range(len(individual_trips)):
    m.addConstr(park_sequence[0, individual_trips[trip], trip] == 1)
    m.addConstr(park_sequence[individual_trips[trip], 0, trip] == 1)
    
# Warm start
m.addConstr(sum(park_sequence[0, destination, trip] for destination in range(num_parks) for trip in range(max_subtrips)) >= 1 + len(individual_trips))

            

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
m.Params.timelimit = 100

# ACTIVATE IF CONSIDERED APPROPRIATE AFTER TUNING 
# m.Params.MIPFocus = 1 # Uncomment for short trip lengths
m.Params.Heuristics = 0.4 # Uncomment for short trip lengths
# m.Params.Cuts = 0 # Uncomment for short trip lengths
# m.Params.Symmetry = 2 
# m.Params.ImproveStartTime = 50

m.optimize()

Set parameter TimeLimit to value 100
Set parameter MIPFocus to value 1
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6602 rows, 99288 columns and 585345 nonzeros
Model fingerprint: 0xd6104eac
Variable types: 63 continuous, 99225 integer (99225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  Objective range  [5e+00, 1e+08]
  Bounds range     [1e+00, 6e+01]
  RHS range        [1e+00, 2e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1722 rows and 14935 columns
Presolve time: 1.41s
Presolved: 4880 rows, 84353 columns, 491871 nonzeros
Variable types: 58 continuous, 84295 integer (84295 binary)

Root relaxation: objective 1.391373e+03, 3436 iterations, 0.50 seconds (0.75 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

In [41]:
# OPTION TO TUNE GUROBI's PARAMETERS TO IMPROVE SOLUTION
# m.Params.timelimit = 200
# m.Params.TuneTimeLimit = 9000
# m.tune()

In [50]:
# Pull out information from the optimized solution

routes_parks = {}
routes_numbers = {}
travel_summary = {}

for trip in range(max_subtrips):
    next_origin = 0
    drive_distance = 0
    flight_count = 0
    travel_time = 0
    time_at_parks = 0
    
    if sum(park_sequence[next_origin,destination,trip].x for origin in range(num_parks) for destination in range(num_parks)) > 0:
        routes_parks[trip] = [park_number_dict[0]]
        routes_numbers[trip] = [0]
        
        while next_origin != -1:
            
            for destination in range(num_parks):
                if park_sequence[next_origin,destination,trip].x == 1:
                    routes_parks[trip].append(park_number_dict[destination])
                    routes_numbers[trip].append(destination)

                    drive_distance += travel[routes_parks[trip][-2]][routes_parks[trip][-1]][0]
                    travel_time += travel[routes_parks[trip][-2]][routes_parks[trip][-1]][1]
                    if (next_origin == 0 and destination in flight_matrix) or (destination == 0 and routes_numbers[trip][-2] in flight_matrix): 
                        flight_count += 1
                    
                    next_origin = destination
                    break

            if next_origin == 0: 
                drive_distance += flight_count
                time_at_parks += length_of_stay*(len(routes_parks[trip])-2)
                next_origin = -1

        travel_summary[trip] = [drive_distance, travel_time, time_at_parks, flight_count]

# print(travel_summary)
# print(routes_parks)
# print(routes_numbers)

In [53]:
# Graphic MAP with trip solution

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

newdf = pd.DataFrame()
df = pd.read_csv('Updated_input_Boston.csv')
df['subtrip'] = [0] * 63

for i in routes_numbers:
    df['subtrip'] = i
    for g in routes_numbers[i]:
        if g == 0:
            newdf = newdf.append(df.loc[[g]])
        else:
            newdf = newdf.append(df.loc[[g]])

fig = px.line_geo(newdf, lon="Longitude", lat="Latitude", hover_name="Park", markers=True, color='subtrip')
fig.show()

C:\Users\cuaco\AppData\Local\Temp\ipykernel_24324\3813840863.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\cuaco\AppData\Local\Temp\ipykernel_24324\3813840863.py:17: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\cuaco\AppData\Local\Temp\ipykernel_24324\3813840863.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\cuaco\AppData\Local\Temp\ipykernel_24324\3813840863.py:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\cuaco\AppData\Local\Temp\ipykernel_24324\3813840863.py:17: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

In [17]:
# After optimizing, write to output file
# Remember to change name of file according to each solution generated

titles = ['Subgroup Details','Total Drive Distance', 'Total Travel Time', 'Total Time at Parks', 'Flight Count']
with open('park_trips_v1.csv', mode='w', newline='', encoding = 'UTF-8') as f:
    
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    writer.writerow(titles)
    for t in travel_summary:
        row = [routes_parks[t], travel_summary[t][0],travel_summary[t][1], travel_summary[t][2], travel_summary[t][3]]
        # write row to file
        writer.writerow(row)